## Тема “Создание признакового пространства”

### Продолжим обработку данных с Твиттера. 

In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

import warnings 
warnings.filterwarnings("ignore")
import os

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

 -	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 -	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 -	Исключим стоп-слова с помощью stop_words='english'.
 -	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [2]:
processed_df = pd.read_pickle('processed_df.pkl')
processed_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


In [3]:
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
stemmed = processed_df['tweet_stemmed'].apply(lambda x: ' '.join(x))
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
lemmatized = processed_df['tweet_lemmatized'].apply(lambda x: ' '.join(x))
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 -	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 -	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 -	Исключим стоп-слова с помощью stop_words='english'.
 -	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [5]:
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.743214,0.391907,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [6]:
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.741146,0.391219,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


3. Натренируем gensim.models.Word2Vec модель на наших данных.
 -	Тренировать будем на токенизированных твитах combine_df['tweet_token']
 -	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
 -	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [7]:
X_train = processed_df['tweet_token'].tolist()

model = Word2Vec(
    X_train, 
    size=200, 
    window=5, 
    min_count=1, 
    sg = 1, 
    hs = 0, 
    negative = 10, 
    workers= 32, 
    seed = 34)

total_examples = len(X_train)
model.train([["hello", "world"]], total_examples=total_examples, epochs=20)

(40, 40)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [8]:
model.most_similar(positive = "dinner")

[('bbq', 0.9603627324104309),
 ('wine', 0.9577217102050781),
 ('brunch', 0.9550037384033203),
 ('cocktails', 0.9508926272392273),
 ('familytime', 0.9494508504867554),
 ('drinks', 0.9493536353111267),
 ('funday', 0.9479431509971619),
 ('cafe', 0.9468299150466919),
 ('funtimes', 0.9464009404182434),
 ('goodtime', 0.9458943009376526)]

In [9]:
model.most_similar(positive = "trump")

[('republican', 0.8947335481643677),
 ('suppoer', 0.8730694055557251),
 ('donald', 0.8691909909248352),
 ('carl', 0.8664813041687012),
 ('protesting', 0.8664739727973938),
 ('blm', 0.8652006387710571),
 ('paladino', 0.8644720315933228),
 ('ally', 0.8642136454582214),
 ('wonthey', 0.8558233976364136),
 ('hillary', 0.8537576794624329)]

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [10]:
model.wv['food']

array([-0.3783253 , -0.19566381,  0.27470955, -0.36893475, -0.29988614,
        0.08885852,  0.59498525,  0.1445426 , -0.07103065, -0.11037741,
       -0.12212033, -0.33117312, -0.33479524,  0.33933228, -0.14648163,
        0.25178567, -0.2334384 , -0.3552802 ,  0.03195114, -0.0926603 ,
        0.17214665,  0.4505072 ,  0.2144498 ,  0.534089  ,  0.31394756,
       -0.40373555,  0.162934  , -0.56925166,  0.25056663,  0.18632835,
       -0.11818361, -0.2871891 , -0.46391973, -0.25628296, -0.36318168,
        0.15690471,  0.13410906, -0.37651455,  0.05039151,  0.09278379,
       -0.0166424 , -0.08223304, -0.60875374,  0.25954258, -0.42595398,
       -0.20292215,  0.08176838,  0.29541948, -0.10598195, -0.09221195,
       -0.20554432,  0.58837765,  0.2202383 , -0.06130573, -0.06748461,
       -0.33819547,  0.0973331 ,  0.00704271,  0.16773668, -0.30569053,
       -0.02727443, -0.35307813, -0.02911951,  0.2017585 , -0.33381584,
       -0.27912474, -0.39538494, -0.14359705, -0.05916009,  0.37

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).

In [11]:
wordvec_df = np.zeros((total_examples, 200))
for i, tweet in enumerate(processed_df['tweet_token']):
    vec = np.zeros((1, 200))
    for word in tweet:
        vec += model.wv[word].reshape((1, 200))
    vec /= len(tweet)
    wordvec_df[i] = vec

In [12]:
wordvec_df.shape

(49159, 200)